In [4]:
# Optimisation of ATS parameters

#imports
import pandas as pd
from importlib import reload
import matplotlib.pyplot as plt
import numpy as np
import pdblp

assets = ['CO1 COMDTY','SP1 INDEX','TY1 COMDTY','HG1 COMDTY', 'NQ1 INDEX', 'GC1 COMDTY']
start_date='20000101'
end_date='20200807'
####functions


def bbloadML(ticker, start_date,end_date):
    
    con = pdblp.BCon(debug=False, port=8194, timeout=5000)
    con.start()
    a = con.bdh(ticker, ['PX_LAST', 'FUT_AGGTE_VOL'],  start_date, end_date )
    a.columns=['close','volume']

    return a

def bestLTMA(prices):
    ret=np.log(prices.close/prices.close.shift(1))
    maxim=-20
    for i in [150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250]:
        ma=prices.close.rolling(i).mean()
        signal=-np.array(ma>prices.close).astype(int)+np.array(ma<prices.close).astype(int)
        dret=signal*ret.shift(-1)
        if sharpe(dret)>maxim:
            best=i
            maxim=sharpe(dret)
    return best

def bestMTMA(prices):
    ret=np.log(prices.close/prices.close.shift(1))
    maxim=-20
    for i in [40, 42, 45, 48, 50, 52, 55, 58, 60, 63, 65, 70, 80, 90, 100]:
        ma=prices.close.rolling(i).mean()
        signal=-np.array(ma>prices.close).astype(int)+np.array(ma<prices.close).astype(int)
        dret=signal*ret.shift(-1)
        if sharpe(dret)>maxim:
            best=i
            maxim=sharpe(dret)
    return best
            
def bestSTMA(prices):
    ret=np.log(prices.close/prices.close.shift(1))
    maxim=-20
    for i in range(13,26,1):
        ma=prices.close.rolling(i).mean()
        signal=-np.array(ma>prices.close).astype(int)+np.array(ma<prices.close).astype(int)
        dret=signal*ret.shift(-1)
        if sharpe(dret)>maxim:
            best=i
            maxim=sharpe(dret)
    return best


def bestSSTMA(prices):
    ret=np.log(prices.close/prices.close.shift(1))
    maxim=-20
    for i in range(4,10,1):
        ma=prices.close.rolling(i).mean()
        signal=-np.array(ma>prices.close).astype(int)+np.array(ma<prices.close).astype(int)
        dret=signal*ret.shift(-1)
        if sharpe(dret)>maxim:
            best=i
            maxim=sharpe(dret)
    return best






def bestSMACD(prices):
    ret=np.log(prices.close/prices.close.shift(1))
    maxim=-20
    for i in range(4,12,1):
        ma1=prices.close.rolling(i).mean()
        for j in range(16,30,1):
            ma2=prices.close.rolling(j).mean()
            signal=-np.array(ma2>ma1).astype(int)+np.array(ma2<ma1).astype(int)
            dret=signal*ret.shift(-1)
            if sharpe(dret)>maxim:
                best=(i,j)
                maxim=sharpe(dret)
    return best


def bestLMACD(prices):
    ret=np.log(prices.close/prices.close.shift(1))
    maxim=-20
    for i in range(40,60,2):
        ma1=prices.close.rolling(i).mean()
        for j in range(150,250,10):
            ma2=prices.close.rolling(j).mean()
            signal=-np.array(ma2>ma1).astype(int)+np.array(ma2<ma1).astype(int)
            dret=signal*ret.shift(-1)
            if sharpe(dret)>maxim:
                best=(i,j)
                maxim=sharpe(dret)
    return best




def bestRSI(prices):
    ret=np.log(prices.close/prices.close.shift(1))
    maxim=-20
    for i in [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]:
        rsi=RSI(prices.close,i)
        for j in [20,25,30,35]:
            signal=np.array(rsi<j).astype(int)-np.array(rsi>100-j).astype(int)
            dret=signal*ret.shift(-1)
            if sharpe(dret)>maxim:
                best=(i,j)
                maxim=sharpe(dret)
    return best


def bestBB(prices):
    ret=np.log(prices.close/prices.close.shift(1))
    maxim=-20
    for i in range(10,35,1):
        ma=prices.close.rolling(i).mean()[i:]
        std=prices.close.rolling(i).std()[i:]
        for j in np.arange(1.,2.,0.1):
            signal=(prices.close[i:]<ma-j*std).astype(int)-(prices.close[i:]>ma+j*std).astype(int)
            dret=signal*ret[i:].shift(-1)
            if sharpe(dret)>maxim:
                best=(i,j)
                maxim=sharpe(dret)
    return best



#RSI function
def RSI(prices, n=14):
    pricediff=prices-prices.shift(1)
    upmove=pd.Series()
    downmove=pd.Series()
    RS=pd.Series()
    RSI=pd.Series()
    upmove=pd.Series([pricediff[i] if pricediff[i]>0 else 0 for i in range(len(pricediff))])
    downmove=pd.Series([-pricediff[i] if pricediff[i]<0 else 0 for i in range(len(pricediff))])
    RS=upmove.rolling(n).mean()/downmove.rolling(n).mean()
    RSI=100-100/(1+RS)
    RSI.index=prices.index
    return RSI




# sharpe for return series (the standard)
def sharpe(logret):
    n = len(logret)
    p=np.exp(logret.sum())**(252/n)-1
    s=np.std(logret)*np.sqrt(252)
    return p/s






########
bestparameterLTMA=dict()
bestparameterMTMA=dict()
bestparameterSTMA=dict()
bestparameterSSTMA=dict()
bestparameterSMACD=dict()
bestparameterLMACD=dict()
bestparameterRSI=dict()
bestparameterBB=dict()

for asset in assets:
    name=asset[:3]
    print("##### Loading "+asset+" and creating features dataframe\n")
    prices = bbloadML(asset, start_date,end_date).dropna()
    n=prices.shape[0]
    prices_train=prices[0:n//2]
    bestparameterLTMA[name]=bestLTMA(prices_train)
    bestparameterMTMA[name]=bestMTMA(prices_train)
    bestparameterSTMA[name]=bestSTMA(prices_train)
    bestparameterSSTMA[name]=bestSSTMA(prices_train)
    bestparameterSMACD[name]=bestSMACD(prices_train)
    bestparameterLMACD[name]=bestLMACD(prices_train)
    bestparameterRSI[name]=bestRSI(prices_train)
    bestparameterBB[name]=bestBB(prices_train)
    
print("LTMA=",bestparameterLTMA)
print("MTMA=",bestparameterMTMA)
print("STMA=",bestparameterSTMA)
print("SSTMA=",bestparameterSSTMA)
print("SMACD=",bestparameterSMACD)
print("LMACD=",bestparameterLMACD)
print("RSI=",bestparameterRSI)
print("BB=",bestparameterBB)





##### Loading CO1 COMDTY and creating features dataframe

##### Loading SP1 INDEX and creating features dataframe

##### Loading TY1 COMDTY and creating features dataframe

##### Loading HG1 COMDTY and creating features dataframe

##### Loading NQ1 INDEX and creating features dataframe

##### Loading GC1 COMDTY and creating features dataframe

LTMA= {'CO1': 160, 'SP1': 230, 'TY1': 210, 'HG1': 170, 'NQ1': 230, 'GC1': 230}
MTMA= {'CO1': 63, 'SP1': 100, 'TY1': 42, 'HG1': 80, 'NQ1': 63, 'GC1': 45}
STMA= {'CO1': 25, 'SP1': 18, 'TY1': 25, 'HG1': 21, 'NQ1': 19, 'GC1': 15}
SSTMA= {'CO1': 5, 'SP1': 9, 'TY1': 9, 'HG1': 4, 'NQ1': 9, 'GC1': 8}
SMACD= {'CO1': (5, 22), 'SP1': (9, 25), 'TY1': (7, 16), 'HG1': (9, 29), 'NQ1': (4, 29), 'GC1': (11, 28)}
LMACD= {'CO1': (54, 240), 'SP1': (54, 210), 'TY1': (54, 150), 'HG1': (56, 150), 'NQ1': (44, 150), 'GC1': (58, 240)}
RSI= {'CO1': (19, 30), 'SP1': (15, 30), 'TY1': (10, 20), 'HG1': (6, 35), 'NQ1': (20, 30), 'GC1': (15, 30)}
BB= {'CO1': (11, 1.9000000000000